---
# Volatility Clustering in the S&P500 and How to Model with GARCH 
**[Nicholas Holloway](https://github.com/nholloway)**

---
### Mission
Stocks and financial time series have familiar jagged edges that crest and fall suddenly, this is in part the result of *volatility clustering* and part of what makes financial timer series difficult to forecast. In this kernel we will cover **heteroskedasticity**, **volatility clustering**, and how to apply the **GARCH** model to forecast the variance of our time series. 

### Table of Contents
1. [Heteroskedasticity](#hetero)
2. [Volatility Clustering](#volatile)
3. [ARCH](#arch)
4. [GARCH](#garch)
5. [GARCH Results](#results)
6. [Model Fit](#fit)
7. [Conclusion](#conclusion)

### Time Series Kernels: 4 of 4 
* [Stationarity, Smoothing, and Seasonality](https://www.kaggle.com/nholloway/stationarity-smoothing-and-seasonality)
* [Deconstructing ARIMA](https://www.kaggle.com/nholloway/deconstructing-arima)
* [Seasonality and SARIMAX](https://www.kaggle.com/nholloway/seasonality-and-sarimax)
* [Volatility Clustering and GARCH](https://www.kaggle.com/nholloway/volatility-clustering-and-garch)

In [1]:
import os
import pandas as pd
import numpy as np
import random
import itertools
from arch import arch_model
from scipy.stats import shapiro
from scipy.stats import probplot
from statsmodels.stats.diagnostic import het_arch
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight') 
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
%matplotlib inline

ModuleNotFoundError: No module named 'arch'

In [3]:
!pip install arch

  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7b09b9cfa5c0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /simple/arch/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7b09b9cfa550>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /simple/arch/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7b09b9cfa470>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',)': /simple/arch/
  Retrying (Retry(total=1, connect=None, read=Non

In [ ]:
path = '../input/individual_stocks_5yr/individual_stocks_5yr'
csvs = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.csv')]

df = pd.DataFrame()
for file in random.sample(range(1, len(csvs)), 8):
    stock_df = pd.read_csv(csvs[file])
    stock_df.index = pd.DatetimeIndex(stock_df.date)
    name = stock_df['Name'].iloc[0]
    df[name] = stock_df['close']

df.plot(figsize=(10, 5), title='Closing Price for 8 Random Stocks')
df.head()

<a id='hetero'></a>
## Heteroskedasticity
We can think about an idealized time series as a set of observations chosen independently and from identical distributions. Understanding this series is as easy as estimating the parameters of the distribution. This is the idea behind stationarity. Once a time series is stationary, we can learn the parameters of the distirbution and forecast, but rarely is it that easy. This kernel is about time series that exhibit heteroskedasticity, specifically conditional heteroskedasticity and the models we use to forecast them.

If we return to  the idealized time series, heteroskedasticity is when the variance of the underlying distribution used to construct our time series changes as a function of time. A common appearance of heterskedasticity is a time series whose variance increases with time, growing outward in a conal pattern. Small amounts of heteroskedasticity can sometimes be masked by a Box-Cox transformation, which makes the data distribution more normal. In our forecasts of the S&P500, not only are our changes in variance too great to mask but the variance at step $x_t$ is conditional on the variance of past time steps, $x_{t-1}$. This is called conditional heteroskedasticity or *volatility clustering*.  

<a id='volatile'></a>
## Volatility Clustering
Volatility clustering is the behavior behind the familiar jagged peaks of financial time series. When we look at the plot above we can see there are periods of high and low volatility. Periods where a positive feedback loop forms and changes in variance give rise to greater changes in variance punctuated by periods of relative calm. The ARCH and GARCH models, which we'll introduce shortly, are designed to model the variance for volatile time series. Note, these models do not forecast estimated values. The ARCH and GARCH models specifically capture the expected variance of the residuals from another model. To forecast financial markets we would use ARCH  and GARCH after applying another model like ARIMA first.

<a id='arch'></a>
## Autoregressive Conditional Heteroskedasticity (ARCH)
If you're familiar with autoregressive (AR) models, ARCH is easy to understand. In an AR model the current value is dependent on $p$ previous timesteps where $p$ lagged values were highly correlated with the current timestep. With ARCH, the variance of the current time step is dependent on $p$ lagged squared standard error terms, with coefficients $\alpha$. The standard error is the difference between the observed and predicted value from another model. 

$$\sigma^2_t=\alpha_0+\alpha_1u^2_{t-1}+\alpha_2u^2_{t-2}+...+\alpha_pu^2_{t-p}$$

If the volatility tends to cluster then recent large squared errors are likely to lead to large variance, and more large squared errors. Like fitting an AR model we can look at PACF and ACF plots to see where our series is autocorrelated and use that to choose a value for our parameter, $p$. If we have an ARCH(1) or a first order ARCH model we assume the series is stationary except for the change in variance. We often use ARCH models in conjunction with another AR or ARMA model where we would pass the ARCH model the squared residuals from the ARMA model. We can also pass ARCH a mean process, as in the case of this kernel where we will pass the percent changes in stock closing value- which is a series that will hover around 0. 

<a id='garch'></a>
## Generalized Autoregressive Conditional Heteroskedasticity (GARCH)
GARCH extends ARCH by allowing the variance to depend on its own lags and the lags of the squared residuals. GARCH can capture greater changes like increasing and decreasing volatility. 

$$\sigma^2_t=\alpha_0+\alpha_1u^2_{t-1}+\alpha_2u^2_{t-2}+...+\alpha_pu^2_{t-p}+\beta_1\sigma^2_{t-1}+...+\beta_p\sigma^2_{t-q}$$

We have two parameters for GARCH(q, p):
* $q$: Number of lag variances
* $p$: Number of lag residual errors 

The parameters we're using here are consistent with the ARCH python library but more often the swapped notation is used. 

GARCH often fits data better than ARCH when there's heteroskedasticity and volatility clustering. We will perform an analysis with both GARCH and ARCH and explain the results but there are lots of variations on the GARCH model that introduce new parameters for the specific behaviors of volatility. 

In [ ]:
stock = 'TDG'
df = pd.read_csv(f'../input/individual_stocks_5yr/individual_stocks_5yr/{stock}_data.csv')
df.index = pd.DatetimeIndex(df.date)
df = df.drop(columns=['open', 'high', 'low', 'volume', 'date', 'Name'])
df['pct_change'] = 100*df['close'].pct_change()
df.dropna(inplace=True)
df['close'].plot(figsize=(10, 5), title=f'{stock} Closing Price 2013-2018')
plt.show()
df['pct_change'].plot(figsize=(10, 5), title=f'{stock} Percent Change in Closing Price')
plt.show()
acf = plot_acf(df['pct_change'], lags=30)
pacf = plot_pacf(df['pct_change'], lags=30)
acf.suptitle(f'{stock} Percent Change Autocorrelation and Partial Autocorrelation', fontsize=20)
acf.set_figheight(5)
acf.set_figwidth(15)
pacf.set_figheight(5)
pacf.set_figwidth(15)
plt.show()

### Ljung-Box Test
Ljung-Box is a test for autocorrelation that we can use in tandem with our ACF and PACF plots. The Ljung-Box test takes our data, optionally either lag values to test, or the largest lag value to consider, and whether to compute the Box-Pierce statistic. Ljung-Box and Box-Pierce are two similar test statisitcs, $Q$, that are compared against a chi-squared distribution to determine if the series is white noise. We might use the Ljung-Box test on the residuals of our model to look for autocorrelation, ideally our residuals would be white noise. 

* $H_o$: The data are independently distributed, no autocorrelation.
* $H_a$: The data are not independently distributed; they exhibit serial correlation.

The Ljung-Box with the Box-Pierce option will return, for each lag, the Ljung-Box test statistic, Ljung-Box p-values, Box-Pierce test statistic, and Box-Pierce p-values. 

If $p<\alpha$ (0.05) we reject the null hypothesis. 

In [ ]:
ljung_res = acorr_ljungbox(df['pct_change'], lags= 40, boxpierce=True)
print(f'Ljung-Box p-values: {ljung_res[1]}')
print(f'\nBox-Pierce p-values: {ljung_res[3]}')

In [ ]:
def ts_plot(residuals, stan_residuals, lags=50):
    residuals.plot(title='GARCH Residuals', figsize=(15, 10))
    plt.show()
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
    ax[0].set_title('GARCH Standardized Residuals KDE')
    ax[1].set_title('GARCH Standardized Resduals Probability Plot')    
    residuals.plot(kind='kde', ax=ax[0])
    probplot(stan_residuals, dist='norm', plot=ax[1])
    plt.show()
    acf = plot_acf(stan_residuals, lags=lags)
    pacf = plot_pacf(stan_residuals, lags=lags)
    acf.suptitle('GARCH Model Standardized Residual Autocorrelation', fontsize=20)
    acf.set_figheight(5)
    acf.set_figwidth(15)
    pacf.set_figheight(5)
    pacf.set_figwidth(15)
    plt.show()

In [ ]:
garch = arch_model(df['pct_change'], vol='GARCH', p=1, q=1, dist='normal')
fgarch = garch.fit(disp='off') 
resid = fgarch.resid
st_resid = np.divide(resid, fgarch.conditional_volatility)
ts_plot(resid, st_resid)
fgarch.summary()

In [ ]:
arch_test = het_arch(resid, maxlag=50)
shapiro_test = shapiro(st_resid)

print(f'Lagrange mulitplier p-value: {arch_test[1]}')
print(f'F test p-value: {arch_test[3]}')
print(f'Shapiro-Wilks p-value: {shapiro_test[1]}')

<a id='garch'></a>
## GARCH Results
---
Here are the results of our GARCH(1,1) model: 

Omega is the baseline variance for the model, so the square root of omega is the standard deviation of returns, 60%. This means that with our mean around 0 we can expect our returns to be 0 with a 60% standard deviation, which is very volatile. Autocorrelation in a model can dampen the volatility and cluster low volatility days but this model is capable of producing very large surges. 

Next, our alpha and beta coefficients. Alpha measures the extent to which a volatility shock today feeds through into next period’s volatility. In our model 9% of the pervious periods volatility will be passed to the next day. Beta is our persistence parameter, if beta is greater than 1 it leads to a postive feedback loop for small shocks that can create runaway volatility. The sum of alpha and beta measures the rate at which our volatility decays and if alpha plus beta equals 1 then our model has persistent volatility and we might want to look at other models like IGARCH (Integrated GARCH).  

According the the text, *Market Risk Analysis* the usual range for alpha in a stable market is $0.05<\alpha<0.1$ and the parameter beta is $0.85<\beta<0.98$. These ranges won't totally apply to forecasting a single stock but it gives us an idea of what kinds of values we should be seeing. 

Next, we have our t-statistics and p-values. T is our estimate divided by the standard error and is used to calculate our p-value. 
The usual null hypothesis is that our coefficient has no effect but we can reject the null if our p value is less than an alpha (0.05). When we begin having models with more parameters it can be helpful to pay attention to the p-values. 

Above we have run some test for goodness of fit that we'll introduce below. All of the tests indicate we should reject the null so we'll continue to search for better model parameters. We can see there is clearly autocorrelation out to at least the 25th lag, and the standardized residuals do not look like white noise. We'll use a gridsearch with farily large range of p and q to find a model that best fits the volatility. 

<a id='fit'></a>
## Model Fit
To measure how well our ARCH/GARCH model fit our data we want to check for: 
1. Autocorrelation in the standardized residuals, we can again use the Ljung-Box test.
2. ARCH effects(conditional heteroskedasticity) in the residuals, we will use Engle's ARCH test on the residuals.
3. Normal distribution in the standardized residuals, we can use the Shapiro-Wilk test, Q-Q plot, and if larger n- the Jarque-Bera test to see if our data approaches normality. .  

### Engle's ARCH Test
* $H_o$: The squared residuals are a sequence of white noise- the residuals are homoscedastic.
* $H_a$: The squared residuals could not be fitted with a linear regression model and exhibit heteroskedasticity. 

In python we will be using the ARCH Engle's test which is a combination of Ljung-Box and a Lagrange Multiplier test to determine if our ARCH model has captured the conditional heteroskedasticity of our time series. We will pass it the array of residuals and a max lag to use.

### Shapiro-Wilks Test
The Shapiro-wilks test evaluates a data sample and quantifies how likely it is that the data was drawn from a Gaussian distribution. 
* $H_o$: The data is normally distributed
* $H_a$: The data is not normally distributed

### Jarque-Bera Test
The Jarque-Bera test is a type of lagrange multiplier test for normality. It usually used for large data sets because other normality tests are not reliable when n is large, Shapiro-Wilk isn't reliable with n more than 2,000. Jarque-Bera specifically mathces skewness and kurtosis to a normal distribution.  
* $H_o$: The data is normally distributed
* $H_a$: The data does not come from a normal distribution

In [ ]:
def gridsearch(data, p_rng, q_rng):
    top_score, top_results = float('inf'), None
    top_models = []
    for p in p_rng:
        for q in q_rng:
            try:
                model = arch_model(data, vol='GARCH', p=p, q=q, dist='normal')
                model_fit = model.fit(disp='off')
                resid = model_fit.resid
                st_resid = np.divide(resid, model_fit.conditional_volatility)
                results = evaluate_model(resid, st_resid)
                results['AIC'] = model_fit.aic
                results['params']['p'] = p
                results['params']['q'] = q
                if results['AIC'] < top_score: 
                    top_score = results['AIC']
                    top_results = results
                elif results['LM_pvalue'][1] is False:
                    top_models.append(results)
            except:
                continue
    top_models.append(top_results)
    return top_models
                
def evaluate_model(residuals, st_residuals, lags=50):
    results = {
        'LM_pvalue': None,
        'F_pvalue': None,
        'SW_pvalue': None,
        'AIC': None,
        'params': {'p': None, 'q': None}
    }
    arch_test = het_arch(residuals, maxlag=lags)
    shap_test = shapiro(st_residuals)
    # We want falsey values for each of these hypothesis tests
    results['LM_pvalue'] = [arch_test[1], arch_test[1] < .05]
    results['F_pvalue'] = [arch_test[3], arch_test[3] < .05]
    results['SW_pvalue'] = [shap_test[1], shap_test[1] < .05]
    return results

In [ ]:
p_rng = range(0,30)
q_rng = range(0,40)
df['dif_pct_change'] = df['pct_change'].diff()
top_models = gridsearch(df['dif_pct_change'], p_rng, q_rng)
print(top_models)

In [ ]:
garch = arch_model(df['pct_change'], vol='GARCH', p=17, q=25, dist='normal')
fgarch = garch.fit(disp='off') 
resid = fgarch.resid
st_resid = np.divide(resid, fgarch.conditional_volatility)
ts_plot(resid, st_resid)
arch_test = het_arch(resid, maxlag=50)
shapiro_test = shapiro(st_resid)
print(f'Lagrange mulitplier p-value: {arch_test[1]}')
print(f'F test p-value: {arch_test[3]}')
print(f'Shapiro-Wilks p-value: {shapiro_test[1]}')
fgarch.summary()

<a id='conclusion'></a>
## Conclusion
---
With parameters GARCH(17, 25) we have a model that passed our Engel ARCH test and whose standardized residuals appear approximately normal. Looking at the residuals and our correlograms this model does not have residuals that appear to be white noise and both the PACF and ACF seem to have a significant coefficient at lag ~22, so we didn't satisfactorily model the conditional variance. Capturing the volatility of a single stock can be very difficult. 

The benefit of this kernel is that it is easy to rerun with a different stock. The first cell shows 8 random stocks and some will certainly be easier to model than others. 'TDG' seemed like a very volitle stock with a few very large dips that clearly persist in the residuals of our final model. If you use this kernel to model another stock it would probably be better to choose one with less volatility and trend. 